In [1]:
import glob
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
import statsmodels.api as sm

In [3]:
homedir = '/home/raghuram/Desktop/radiomics/TEXTURES/'
os.chdir(homedir)

In [4]:
# List all the data sources
t1_expt = 'expt_t1w.csv'
t2_expt = 'expt_t2w.csv'
t1ce_expt = 'expt_t1ce.csv'
t2f_expt = 'expt_t2f.csv'

In [155]:
def preprocess_data(expt_list, expt_number):
    df = pd.read_csv(expt_list)
    df = df[df['experiment_number'] == expt_number]
    df.dropna(inplace=True)
    volume = df['VOLUME_WT']
    df.drop(columns=['mag_field_strength', 'scanner_model', 'scanner_manufacturer',
                    'parameters_Ng', 'parameters_Algo', 'parameters_Scale', 'mat_file_name',
                    'sequence_name', 'repetition_time', 'excitation_time', 'scale', 'algo', 'ng',
                    'flip_angle', 'experiment_number', 'ID', 'VOLUME_ET','VOLUME_NET', 'VOLUME_ED', 
                     'VOLUME_TC', 'VOLUME_WT', 'VOLUME_BRAIN'], inplace=True)
    
    X = np.array(df)
    return X, volume
    

In [172]:
results_folder = '/home/raghuram/Desktop/radiomics/TEXTURES/results/t2w/vol_corr'

In [174]:
for expt_number in range(1, 26):
    X, volume = preprocess_data(t2_expt, expt_number)
    
    assert(np.shape(X)[0] == np.shape(volume)[0])
 
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = sm.add_constant(X)
    
    model = sm.OLS(volume, X).fit()
    
    with open(os.path.join(results_folder, 't2w_expt'+'_'+str(expt_number)+'_'+'volume.csv'), 'w') as csvfile:
        csvfile.write(model.summary().as_csv())
    